# Test Word2vec

In [1]:
import numpy as np
import pandas as pd
import pickle
import gensim, logging
import gensim.models.keyedvectors as word2vec
import matplotlib.pyplot as plt

%matplotlib inline

# Data

In [2]:
# Importing prepared dataset
import pickle
input = open('my_corpus-21-sql.pql', 'rb')
obj = pickle.load(input)
input.close()

txt = obj['txt']
corpus = obj['corpus']
words_list = obj['words_list']
all_words = obj['all_words']
print(corpus[1])

світовий пандемія принести невизначеність вимушений реагувати надшвидко маючи право помилка бізнес винаходити модель благодійний адаптуватися кризовий умова пропонуючи підхід рішення проєкт травень директорка виступити віртуальний саміт благодійництво тема фандрайзинг специфіка ковіднога долучитися дискусіїй секція благодійник бізнес адаптивність криза запрошувати прийняти саміт участь безкоштовний реєстрація посилання


In [3]:
txt.reset_index(drop=True,inplace=True)

## Splitting

In [4]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
w1, w2 = train_test_split(words_list, test_size=0.5, random_state=10)

# W2V

In [5]:
model = word2vec.KeyedVectors.load_word2vec_format('ubercorpus.lowercased.lemmatized.word2vec.300d.txt', binary=False)

In [6]:
words = list(model.key_to_index)

In [7]:
def txt_embed(words, corpus):
    x_sent_embed = []
    count_words, count_non_words = 0, 0  
    
    # recover the embedding of each sentence with the average of the vector that composes it
    # sent - sentence
    for sent in corpus:
        # average embedding of all words in a sentence
        sent_embed = []
        for word in sent:
            try:
                # if word is present in the dictionary - add its vector representation
                count_words += 1
                sent_embed.append(model[word])
            except KeyError:
                # if word is not in the dictionary - add a zero vector
                count_non_words += 1
                sent_embed.append([0] * 300)
        
        # add a sentence vector to the list
        x_sent_embed.append(np.mean(sent_embed, axis=0).tolist())
                    
    print(count_non_words, "out of", count_words, "words were not found in the vocabulary.")
    
    return x_sent_embed

In [8]:
x1 = txt_embed(words, w1)
x2 = txt_embed(words, w2)

C:\Users\kater\anaconda3\envs\covid\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\kater\anaconda3\envs\covid\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


6488 out of 143479 words were not found in the vocabulary.
6501 out of 159010 words were not found in the vocabulary.


In [10]:
embed1 = []
for i,v in enumerate(x1): 
    if np.any(np.isnan(v)) != True:
        embed1.append(np.array(v))
    else:
        
        embed1.append(np.array([0] * 300))
        print(i)
em1 = np.vstack(embed1)

7
192
355
571
846
851
1054
1219


In [11]:
embed2 = []
for i,v in enumerate(x2): 
    if np.any(np.isnan(v)) != True:
        embed2.append(np.array(v))
    else:
        embed2.append(np.array([0] * 300))
        print(i)
em2 = np.vstack(embed2)

264
327


# K-means

In [13]:
# Number of Clusters
k = 9

In [45]:
# K-means clustering
from sklearn.cluster import KMeans
km1 = KMeans(n_clusters = k).fit(em1)
km2 = KMeans(n_clusters = k).fit(em2)

# Comparing Clustering Algorithms
from sklearn.metrics.cluster import adjusted_rand_score
(adjusted_rand_score(km1.labels_.tolist(),km2.labels_.tolist()).round(3))*100

-0.1

## W2V AgglomerativeClustering

In [44]:
from sklearn.cluster import AgglomerativeClustering
hc1 = AgglomerativeClustering(n_clusters = k, affinity = 'euclidean', linkage = 'ward').fit_predict(em1)
hc2 = AgglomerativeClustering(n_clusters = k, affinity = 'euclidean', linkage = 'ward').fit_predict(em2)

# Comparing Clustering Algorithms
(adjusted_rand_score(hc1,hc2).round(3))*100

0.0

## MeanShift

In [43]:
from sklearn.cluster import MeanShift
mshift1 = MeanShift(bandwidth=20).fit(em1)
mshift2 = MeanShift(bandwidth=20).fit(em2)

# Comparing Clustering Algorithms
(adjusted_rand_score(mshift1.labels_,mshift2.labels_).round(3))*100

-0.6

In [31]:
len(set(mshift1.labels_))

9

## HDBSCAN

In [42]:
import hdbscan
hdb1 = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=7).fit(em1)
hdb2 = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=7).fit(em2)

# Comparing Clustering Algorithms
(adjusted_rand_score(hdb1.labels_,hdb2.labels_).round(3))*100

1.5

In [26]:
len(set(hdb2.labels_))

9